In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../input/pokemon_train.npy
../input/pokemon_test.npy


In [2]:
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [3]:
TRAIN_PATH = "../input/pokemon_train.npy"
TEST_PATH = "../input/pokemon_test.npy"


In [4]:
def preprocess_data():
    train_data = np.load(TRAIN_PATH)
    test_data = np.load(TEST_PATH)

    # random
    rng = np.random.RandomState(SEED)
    indices = np.arange(len(train_data))
    rng.shuffle(indices)
    train_data = train_data[indices]

    val_data = train_data[:100]

    train_data = train_data[100:]

    x_train = train_data[:,1:]
    y_train = train_data[:,0]

    x_val = val_data[:,1:]
    y_val = val_data[:,0]

    return x_train, y_train, x_val, y_val, test_data

In [5]:
def build_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(128, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(128, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    print(model.summary())
    return model

In [6]:
SEED = 113
model = build_model()
x_train, y_train, x_val, y_val, x_test = preprocess_data()
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

x_train = np.reshape(x_train, (x_train.shape[0], 128,128,3))
x_val = np.reshape(x_val, (x_val.shape[0], 128,128,3))

train_datagen.fit(x_train)
validation_datagen.fit(x_val)

model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=20),
                  steps_per_epoch=100, 
                  epochs=30,
                  validation_data=validation_datagen.flow(x_val, y_val, batch_size=20),
                  validation_steps=20)
    


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 128)       147584    
__________

In [7]:
x_test = x_test.reshape(-1, 128, 128, 3)
x_test = x_test / 255
predict_labels = model.predict_classes(x_test, batch_size=32)
print(predict_labels)
predict_label_csv = np.hstack([(np.arange(predict_labels.shape[0])+1).reshape([-1, 1]), predict_labels.reshape([-1, 1])])
np.savetxt('predict_label.csv', predict_label_csv, delimiter = ',', header='Id,Category')

[4 4 4 3 3 4 1 3 2 2 1 0 0 2 2 2 3 2 1 0 1 4 4 2 4 3 0 3 0 4 1 1 1 1 3 2 0
 3 3 1 1 4 2 2 1 1 4 3 2 1 1 3 0 3 0 4 4 3 2 1 0 1 3 0 0 2 4 2 3 0 1 0 3 4
 2 0 3 2 2 1 1 3 1 2 1 2 3 3 4 3 2 2 0 1 2 2 0 3 4 3 1 4 2 3 2 3 4 2 1 4 0
 1 0 0 2 2 3 1 3 2 3 2 1 3 0 2 3 2 1 2 4 4 0 0 0 0 1 3 1 0 2 1 1 4 1 2 0 2
 3 1 3 3 1 3 1 0 1 4 2 0 4 2 1 0 0 0 2]
